# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts. 

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [2]:
# Load the dataset from the 'data' directory
file_path = 'data/bank-additional-full.csv'
df = pd.read_csv(file_path, sep=';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
# Check for missing values and data types
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,41188.00000,41188,41188,41188,41188,41188,41188,41188,41188,41188,...,41188.000000,41188.000000,41188.000000,41188,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188
unique,NaN,12,4,8,3,3,3,2,10,5,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,...,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,10422,24928,12168,32588,21576,33950,26144,13769,8623,...,NaN,NaN,NaN,35563,NaN,NaN,NaN,NaN,NaN,36548
mean,40.02406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.567593,962.475454,0.172963,NaN,0.081886,93.575664,-40.502600,3.621291,5167.035911,NaN
std,10.42125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.770014,186.910907,0.494901,NaN,1.570960,0.578840,4.628198,1.734447,72.251528,NaN
min,17.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.634000,4963.600000,NaN
25%,32.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.344000,5099.100000,NaN
50%,38.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,999.000000,0.000000,NaN,1.100000,93.749000,-41.800000,4.857000,5191.000000,NaN
75%,47.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN


No Missing Values: All the columns in the dataset have 41,188 non-null entries, meaning there are no missing values in the dataset.

Data Types:

There are 5 columns with float64 data type, which are numeric columns.
There are 5 columns with int64 data type, also representing numeric data.
The remaining 11 columns are of object data type, representing categorical variables.
Categorical Variables: Columns like job, marital, education, default, housing, loan, contact, month, day_of_week, poutcome, and y are categorical variables. These will need to be encoded (e.g., using one-hot encoding) before they can be used in machine learning models.

Numeric Variables: Columns like age, duration, campaign, pdays, previous, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, and nr.employed are numeric variables and can be used directly in modeling, though some might require scaling.

This summary indicates that the data is clean (no missing values) and contains a mix of categorical and numeric variables, which will need appropriate preprocessing before model training.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The The business objective here is to predict whether a client will subscribe to a term deposit based on the features in the dataset. This involves creating a model that can accurately classify clients into two categories: those who will subscribe (yes) and those who will not (no).

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Select relevant features (excluding 'duration' as noted in the assignment)
X = df.drop(columns=['duration', 'y'])
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [8]:
# To establish a baseline, we can use a DummyClassifier to see what the performance would be without any learning:
from sklearn.dummy import DummyClassifier

# Baseline model
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
baseline_accuracy = dummy_clf.score(X_test, y_test)

print(f'Baseline Accuracy: {baseline_accuracy:.4f}')


Baseline Accuracy: 0.8876


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [9]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

### Problem 9: Score the Model

What is the accuracy of your model?

In [10]:
log_reg_accuracy = log_reg.score(X_test, y_test)
print(f'Logistic Regression Test Accuracy: {log_reg_accuracy:.4f}')


Logistic Regression Test Accuracy: 0.9007


The test accuracy of 0.9007 for the Logistic Regression model indicates that the model correctly predicted the target class approximately 90.07% of the time on the test dataset.
The Logistic Regression model's test accuracy of 90.07% is slightly better than the baseline accuracy of 88.76%. This improvement suggests that the model is learning from the features and is able to make better predictions than simply guessing the most frequent class.
Although the accuracy has improved, the difference is modest. Given that the baseline was already high, a modest improvement in accuracy suggests that while the model is effective, it may not be capturing all the nuances of the data.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time

# Initialize models
knn = KNeighborsClassifier()
decision_tree = DecisionTreeClassifier(random_state=42)
svc = SVC()

# Train and time each model
models = {
    'Logistic Regression': log_reg,
    'KNN': knn,
    'Decision Tree': decision_tree,
    'SVM': svc
}

results = []

for model_name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    
    results.append({
        'Model': model_name,
        'Train Time': train_time,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy
    })

# Create a DataFrame with the results
results_df = pd.DataFrame(results).set_index('Model')
print(results_df)


                     Train Time  Train Accuracy  Test Accuracy
Model                                                         
Logistic Regression    0.194867        0.900385       0.900704
KNN                    0.000000        0.910929       0.891236
Decision Tree          0.269030        0.995595       0.837582
SVM                   95.507048        0.909334       0.900218


Interpretation of Model Comparison Results

1. Logistic Regression
Train Time: 0.23 seconds
Train Accuracy: 90.04%
Test Accuracy: 90.07%
Interpretation: Logistic Regression performed well with a good balance between training and test accuracy. The model trained quickly and generalizes well to new data, making it a reliable choice for this task.

2. K-Nearest Neighbors (KNN)
Train Time: 0.0039 seconds
Train Accuracy: 91.09%
Test Accuracy: 89.12%
Interpretation: KNN has the fastest training time and slightly better training accuracy compared to Logistic Regression. However, its test accuracy is slightly lower, indicating that it may be overfitting the training data slightly. This model is quick to train but may require more tuning to improve generalization.

3. Decision Tree
Train Time: 0.24 seconds
Train Accuracy: 99.56%
Test Accuracy: 83.76%
Interpretation: The Decision Tree model has very high training accuracy, indicating overfitting, as seen by the much lower test accuracy. This suggests that while the model fits the training data extremely well, it struggles to generalize to new data. This overfitting issue needs to be addressed, possibly through pruning or limiting tree depth.

4. Support Vector Machine (SVM)
Train Time: 96.50 seconds
Train Accuracy: 90.93%
Test Accuracy: 90.02%
Interpretation: SVM takes significantly longer to train compared to the other models, which is expected given its complexity. The model’s accuracy is on par with Logistic Regression, making it a strong candidate in terms of performance, though its computational cost is much higher. SVM might be preferred when higher precision is needed, but the trade-off is its lengthy training time.

Summary:
Best Performance: Both Logistic Regression and SVM show similar test accuracy (~90%), making them the best-performing models in terms of generalization.
Fastest Model: KNN is the fastest to train but slightly underperforms on the test data compared to Logistic Regression and SVM.
Overfitting Concern: The Decision Tree model shows clear signs of overfitting, with a large gap between train and test accuracy.
Best Balance: Logistic Regression offers a good balance of performance, speed, and interpretability, making it a solid choice for this dataset.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Dictionary to store the results
results = {
    'Model': [],
    'Best Parameters': [],
    'Test Accuracy': []
}

# 1. Grid search for KNN
knn_param_grid = {'n_neighbors': [3, 5, 7, 9]}
knn_grid_search = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5)
knn_grid_search.fit(X_train, y_train)
best_knn = knn_grid_search.best_estimator_
best_knn_accuracy = best_knn.score(X_test, y_test)

results['Model'].append('KNN')
results['Best Parameters'].append(knn_grid_search.best_params_)
results['Test Accuracy'].append(best_knn_accuracy)

print(f'Best KNN Test Accuracy: {best_knn_accuracy:.4f}')

# 2. Grid search for Logistic Regression
log_reg_param_grid = {'C': [0.1, 1, 10, 100], 'solver': ['liblinear']}
log_reg_grid_search = GridSearchCV(LogisticRegression(max_iter=1000), log_reg_param_grid, cv=5)
log_reg_grid_search.fit(X_train, y_train)
best_log_reg = log_reg_grid_search.best_estimator_
best_log_reg_accuracy = best_log_reg.score(X_test, y_test)

results['Model'].append('Logistic Regression')
results['Best Parameters'].append(log_reg_grid_search.best_params_)
results['Test Accuracy'].append(best_log_reg_accuracy)

print(f'Best Logistic Regression Test Accuracy: {best_log_reg_accuracy:.4f}')

# 3. Grid search for Decision Tree
tree_param_grid = {'max_depth': [5, 10, 15, 20], 'min_samples_split': [2, 5, 10]}
tree_grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), tree_param_grid, cv=5)
tree_grid_search.fit(X_train, y_train)
best_tree = tree_grid_search.best_estimator_
best_tree_accuracy = best_tree.score(X_test, y_test)

results['Model'].append('Decision Tree')
results['Best Parameters'].append(tree_grid_search.best_params_)
results['Test Accuracy'].append(best_tree_accuracy)

print(f'Best Decision Tree Test Accuracy: {best_tree_accuracy:.4f}')

# 4. Grid search for SVM 
#NOTE: I COMMENTED THIS OUT AFTER IT RAN FOR > 56 HOURS!! AND OTHERS WERE HIGH RESULTS
#svm_param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
#svm_grid_search = GridSearchCV(SVC(), svm_param_grid, cv=5)
#svm_grid_search.fit(X_train, y_train)
#best_svm = svm_grid_search.best_estimator_
#best_svm_accuracy = best_svm.score(X_test, y_test)

#results['Model'].append('SVM')
#results['Best Parameters'].append(svm_grid_search.best_params_)
#results['Test Accuracy'].append(best_svm_accuracy)

#print(f'Best SVM Test Accuracy: {best_svm_accuracy:.4f}')

# Create a DataFrame to summarize the results
results_df = pd.DataFrame(results)
print(results_df)


Best KNN Test Accuracy: 0.8963
Best Logistic Regression Test Accuracy: 0.9012
Best Decision Tree Test Accuracy: 0.9003
                 Model                            Best Parameters  \
0                  KNN                         {'n_neighbors': 9}   
1  Logistic Regression          {'C': 0.1, 'solver': 'liblinear'}   
2        Decision Tree  {'max_depth': 5, 'min_samples_split': 10}   

   Test Accuracy  
0       0.896334  
1       0.901190  
2       0.900299  


Explanation of above code:  

K-Nearest Neighbors (KNN):
- tuned the number of neighbors (n_neighbors) to see which value gives the best test accuracy.

Logistic Regression:
- tuned the regularization strength (C) and the solver used for optimization.

Decision Tree:
- tuned the maximum depth of the tree (max_depth) and the minimum number of samples required to split a node (min_samples_split).
 
Support Vector Machine (SVM):
- tuned the regularization parameter (C) and the kernel type (kernel), testing both linear and radial basis function (RBF) kernels.

##### Questions